In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import re

## Login to  Linkedin

In [39]:
# Login to Linkedin
# Open chrome and direct to linkedin login page
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")
driver.implicitly_wait(10)
url = 'https://www.linkedin.com/login'
driver.get(url)

# Extract credentital information
credential = open('login_credential.txt')
line = credential.readlines()
username = line[0]
password = line[1]

# Fill form & login
email_field = driver.find_element(By.ID, 'username')
email_field.send_keys(username)
sleep(1.5)
password_field = driver.find_element(By.NAME, 'session_password')
password_field.send_keys(password)
sleep(1.5)
login_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
login_button.click()
sleep(1)

C:\Users\Tung Lam\AppData\Local\Temp\ipykernel_3780\842975801.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")


In [40]:
# Search for the profiles
driver.maximize_window()
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
search_query = input('What profile do you want to scrape? ')
search_field.send_keys(search_query)
search_field.send_keys(Keys.RETURN)

In [41]:

people_button = driver.find_element(By.CSS_SELECTOR, '[aria-label="People"]')
people_button.click()

In [42]:
# Retrieve all profile url in current page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', href = re.compile("/in/"))
    all_profile_url = []
    for profile in profiles:
        profile_url = profile['href'][0:profile['href'].find('?')]
        if profile_url not in all_profile_url:
            all_profile_url.append(profile_url)
    return all_profile_url

In [43]:
# Retrieve profile url in multiple pages
def GetURLonPages():
    number_of_page = int(input('Number of page you want to crawl: '))
    profile_urls = []
    for page in range(number_of_page):
        urls_one_page = GetURL()
        profile_urls = profile_urls + urls_one_page
        sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        driver.find_element(By.CSS_SELECTOR, '[aria-label="Next"]').click()
        sleep(1)
    return profile_urls

profile_urls = GetURLonPages()
print(profile_urls)

['https://www.linkedin.com/in/nguyen-van-long-73991710b', 'https://www.linkedin.com/in/ACoAAB47cUkBj129UY0d15DbNWlflWCz04ubOXM', 'https://www.linkedin.com/in/do-binh-10993757', 'https://www.linkedin.com/in/dung-nguyen-27637a175', 'https://www.linkedin.com/in/ACoAAAAZ_ssBRAJfOZd4ipTcrzNbgglLKiLJeCY', 'https://www.linkedin.com/in/manhitv', 'https://www.linkedin.com/in/minh-ha-phan-670716185', 'https://www.linkedin.com/in/duong-son-22aaab7a', 'https://www.linkedin.com/in/linh-c-le', 'https://www.linkedin.com/in/ACoAAAS-7iABHzeQUCqTF-Pk-EZDvSMzLYwiWr4', 'https://www.linkedin.com/in/chilevan', 'https://www.linkedin.com/in/cuongvn08', 'https://www.linkedin.com/in/duy-truong-71457930']


In [46]:
import csv
with open('linkedin-profile.csv', 'w', newline = '', encoding="utf-8") as fhand:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(fhand, delimiter=',', lineterminator='\n', fieldnames=headers)
    writer.writeheader()

    for profile_url in profile_urls:
        driver.get(profile_url)
        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        info_div = page_source.find('div', class_="mt2 relative")
        name = info_div.h1.string.strip()
        title = info_div.find('div', class_="text-body-medium break-words").string.strip()
        location = info_div.find('span', class_="text-body-small inline t-black--light break-words").string.strip()
        print(name, title, location)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:profile_url})

Nguyen Van Long Data Engineer at Giaohangtietkiem.vn Vietnam
Lưu Thị Hiền Hiền Go big or go home! Vietnam
Do Binh Senior Data Scientist at MSB Vietnam
Dung Nguyen Senior Data Scientist at Zalo Vietnam
Triều (Charles) Nguyễn Founder at PrimeData.ai - Science-first Customer Data & eXperience Platform Vietnam
Manh Nguyen Senior Data Scientist at Vingroup Hanoi, Hanoi, Vietnam
Minh Ha Phan Data Analyst at One Mount Group Vietnam
Duong Son Data Scientist at Lazada Group Vietnam
Linh Le Data Analyst at One Mount Group Ho Chi Minh City, Vietnam
Thanh Luu Head of Data Product at GalaxyOne (Sovico Group) Vietnam
Chi Le Van Data Engineer at PrimeData Science-first Marketing Platform Ho Chi Minh City, Vietnam
Cuong Barry Senior Data Scientist Vietnam
Duy Truong Data scientist manager at HEINEKEN Vietnam Vietnam
